In [2]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D,Reshape,Dropout,Lambda
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
#import tensorflow.initializers as initializers 

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
# from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [106]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
# file_names = ['AgiaParaskevi','Aristotelous','Elefsina',
#               'Lykovrisi','Marousi','NeaSmirni','Peristeri',
#               'Pireus','Thrakomakedones']
file_names = [
              'Marousi',
              'Aristotelous',
              'NeaSmirni',
              'AgiaParaskevi',
              'Pireus',
              'Peristeri'
             ]

# file_names = [
#               'Marousi'
#              ]

file_names_nogrid = [
                'Thrakomakedones',
                'Elefsina'
            ]

grid = {'Marousi': ['Lykovrisi','Peristeri','AgiaParaskevi']
        ,'Aristotelous': ['Marousi','Peristeri','NeaSmirni']
        ,'AgiaParaskevi': ['Marousi','NeaSmirni','Koropi']
        ,'NeaSmirni': ['Aristotelous','Pireus','AgiaParaskevi']
        ,'Pireus': ['Elefsina','Peristeri','NeaSmirni']
        ,'Peristeri':['Thrakomakedones','Marousi','Pireus']
       }


grid2 = {'Marousi': ['Lykovrisi','Aristotelous']
        ,'Aristotelous': ['Marousi','Peristeri']
        ,'AgiaParaskevi': ['Marousi','Koropi']
        ,'NeaSmirni': ['Aristotelous','Pireus']
        ,'Pireus': ['Elefsina','NeaSmirni']
        ,'Peristeri':['Thrakomakedones','Marousi']
       }


file_names_mock = [
                'Elefsina'
            ]

In [4]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [5]:
def get_time(tmp):
    return tmp[11:13]

In [6]:
def get_season(tmp):
    if tmp == 'Spring':
        return '1'
    elif tmp == 'Spring/Summer':
        return '2'
    elif tmp == 'Summer':
        return '3'
    elif tmp == 'Summer/Autumn':
        return '4'
    elif tmp == 'Autumn':
        return '5'
    elif tmp == 'Autumn/Winter':
        return '6'
    elif tmp == 'Winter/Spring':
        return '7'
    else:
        return '8'

In [7]:
def get_winddir(tmp):
    if tmp == 'N':
        return '1'
    elif tmp == 'NNE':
        return '2'
    elif tmp == 'NE':
        return '3'
    elif tmp == 'ENE':
        return '4'
    elif tmp == 'E':
        return '5'
    elif tmp == 'ESE':
        return '6'
    elif tmp == 'SE':
        return '7'
    elif tmp == 'SSE':
        return '8'
    elif tmp == 'S':
        return '9'
    elif tmp == 'SSW':
        return '10'
    elif tmp == 'SW':
        return '11'
    elif tmp == 'WSW':
        return '12'
    elif tmp == 'W':
        return '13'
    elif tmp == 'WNW':
        return '14'
    elif tmp == 'NW':
        return '15'
    else:
        return '16'

In [8]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')
#agiaparaskevi = pd.read_csv('CleanFiles/AgiaParaskeviExtract.csv', sep=',')

In [9]:
# T = 72
# D = 1
# X = []
# Y = []
# for t in range(len(neasmirni) - T):
#     x = neasmirni[t:t+T]
#     X.append(x)
#     y = neasmirni[t:t+T]
#     Y.append(y)

In [10]:
def convert_pm10_index(predicts, test):
    for i in range(len(predicts)):
        for j in range(len(predicts[i])):
            predicts[i][j] = pm10_index(predicts[i][j])
            test[i][j] = pm10_index(test[i][j])

In [11]:
def calc_pm10_accuracy(predicts, test, window):
    tot_val = len(predicts) * window 
    tot_comp = 0
    for i in range(len(predicts)):
        for j in range(window):
            if(abs(predicts[i][j] - test[i][j]) == 1):
                tot_comp =  tot_comp + 0.5
            elif(abs(predicts[i][j] == test[i][j])):
                tot_comp =  tot_comp + 1
    return (100*tot_comp)/tot_val

In [12]:
def create_xy(series, series2, window_size, prediction_horizon, shuffle = False):
    x = []
    y = []
    for i in range(0, len(series)):
        if len(series[(i + window_size):(i + window_size + prediction_horizon)]) < prediction_horizon:
            break
        x.append(np.array(series[i:(i + window_size)]))
        y.append(np.array(series2[(i + window_size):(i + window_size + prediction_horizon)]))
    x = np.array(x)
    y = np.array(y)
    return x,y

In [13]:
parameters = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    'boosting_type': 'dart'
}

In [14]:
def smape(actual, predicted):
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

In [15]:
def gru(units,drop,features, window, horizon):
    encoder_inputs = layers.Input(shape=(window,features))
    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    out = layers.TimeDistributed(Dense(1))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop())
    return model

In [16]:
def seq2seq(window, horizon, units,bn,drop,feautures,channel=1):
    encoder_inputs = layers.Input(shape=(window,feautures))

    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    if bn:
        encoder_states=layers.BatchNormalization()(encoder_states)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    decoder_gru = GRU(units, dropout=drop, return_sequences=True, return_state=False)
    decoder = decoder_gru(decoder, initial_state=encoder_states)
    
    out = layers.TimeDistributed(Dense(channel,use_bias=True))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [17]:
def gatedDNN(units,features, horizon):
#     model = Sequential()
#     model.add(layers.Input(shape=(72,features)))
#     model.add(layers.Dense(2, activation='relu'))
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

    model = Sequential([
    Lambda(lambda x: x[:, -1:, :]),
    Dense(units, activation='relu',use_bias=True),
    Dense(horizon*1),
    Reshape([horizon, 1])
    ])
    model.add(LeakyReLU(alpha=0.01))
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [18]:
def lstm(features, window, horizon):
    model = Sequential([
    LSTM(10, input_shape=(window, features), return_sequences=True),
    Dropout(0.5),
    LSTM(10, return_sequences=True),
    Dense(horizon),
    Reshape([horizon, 1])
    ])
    
    model.compile(optimizer=RMSprop(), loss='mse', metrics=['mean_squared_error'])
    return model

In [19]:
def lightgbm():
    model = lgb.LGBMRegressor(first_metric_only = True)
    return model

In [92]:
def loadToModel(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df2 = df[['pm10']]
        

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,7,1)
#         model = gru(5,0.5,7, 72, 48)
#         model = lstm(5, window, horizon)
#         model = lightgbm()
        model = gatedDNN(100,7, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test, model, scaler, scaler1, tmpDf

In [21]:
def loadToModelGrid2(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        characteristics = 10
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df1 = pd.read_csv(pathFiles + grid2[fileName][0] + formating, sep=',')
        df2 = pd.read_csv(pathFiles + grid2[fileName][1] + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df1['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df2['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df['windir1'] = df1['forecast_windDirection']
        df['pm101'] = df1['pm10']
        df['windir2'] = df2['forecast_windDirection']
        df['pm102'] = df2['pm10']
        
        df2 = df[['pm10']]
        

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,15,1)
#         model = gru(5,0.5,15, 72, 48)
#         model = lstm(15, window, horizon)
#         model = lightgbm()
        model = gatedDNN(100,characteristics, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test

In [22]:
def loadToModelGrid3(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        characteristics = 12
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df1 = pd.read_csv(pathFiles + grid[fileName][0] + formating, sep=',')
        df2 = pd.read_csv(pathFiles + grid[fileName][1] + formating, sep=',')
        df3 = pd.read_csv(pathFiles + grid[fileName][2] + formating, sep=',')
#         df4 = pd.read_csv(pathFiles + grid[fileName][3] + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df1['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df2['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df3['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df['windir1'] = df1['forecast_windDirection']
        df['pm101'] = df1['pm10']
        df['windir2'] = df2['forecast_windDirection']
        df['pm102'] = df2['pm10']
        df['windir3'] = df3['forecast_windDirection']
        df['pm103'] = df3['pm10']
        
        df2 = df[['pm10']]
        

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10','windir1','pm101','windir2','pm102','windir3','pm103']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
#         model = seq2seq(72,48,50,True,0.5,15,1)
#         model = gru(5,0.5,15, 72, 48)
#         model = lstm(15, window, horizon)
#         model = lightgbm()
        model = gatedDNN(300,characteristics, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

    return predictions, y_test, model

In [126]:
def mockPredictions(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in file_names_mock:
        window = 72 
        horizon = 48
        accuracy = 0
        characteristics = 12
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        
        df2 = df[['pm10']]
        
        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']],df2,window, horizon)
        accuracy = 0.0
        

        return X[0]

In [122]:
predicts, test, model1 = loadToModelGrid3(file_names, files_format, files_path, files_postfix, parameters)
#predicts, test = loadToModel(file_names, files_format, files_path, files_postfix, parameters)

Marousi begins prediction
Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
916/916 [==============================] - 2s 2ms/step - loss: 0.9992 - mean_squared_error: 0.9992
Epoch 2/2
916/916 [==============================] - 3s 3ms/step - loss: 0.0335 - mean_squared_error: 0.0335
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) an

Pireus begins prediction
Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
916/916 [==============================] - 2s 1ms/step - loss: 1.0127 - mean_squared_error: 1.0127
Epoch 2/2
916/916 [==============================] - 2s 3ms/step - loss: 0.0630 - mean_squared_error: 0.0630
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and

In [39]:
#predicts[10000]

In [26]:
#test[10000]

In [27]:
print(np.amin(predicts))
print(np.amin(test))
print(np.amax(predicts))
print(np.amax(test))

-2.8455806
0.0
232.04393
802.0


In [28]:
convert_pm10_index(predicts, test)

In [29]:
print(calc_pm10_accuracy(predicts,test,48))

70.56005012357656


In [30]:
model1.save('models/model1.h5')

In [93]:
predicts, test, model2, sc1, sc2, tmpDf = loadToModel(file_names_nogrid, files_format, files_path, files_postfix, parameters)

Thrakomakedones begins prediction
Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
916/916 [==============================] - 2s 1ms/step - loss: 2.2446 - mean_squared_error: 2.2446
Epoch 2/2
916/916 [==============================] - 2s 3ms/step - loss: 0.0316 - mean_squared_error: 0.0316
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY

In [94]:
#predicts[10000]

In [95]:
#test[10000]

In [96]:
print(np.amin(predicts))
print(np.amin(test))
print(np.amax(predicts))
print(np.amax(test))

3.5303736
0.0
157.753
684.0


In [97]:
convert_pm10_index(predicts, test)

In [98]:
print(calc_pm10_accuracy(predicts,test,48))

71.66660892061073


In [54]:
model2.save('models/model2.h5')

In [59]:
from keras.models import load_model

In [72]:
new_model1 = load_model('models/model1.h5')
new_model2 = load_model('models/model2.h5')

In [127]:
tmpDs = mockPredictions(file_names_nogrid, files_format, files_path, files_postfix, parameters)

In [134]:
tmpDs.shape

(72, 7)

In [117]:
#tmpDsX, tmpDsY = create_xy(tmpDs[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']], tmpDs[['pm10']], 72, 48)

In [135]:
new_model2.predict(tmpDs)

ValueError: Exception encountered when calling layer "lambda_14" (type Lambda).

Index out of range using input dim 2; input has only 2 dims for '{{node sequential_14/lambda_14/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=5, ellipsis_mask=0, end_mask=7, new_axis_mask=0, shrink_axis_mask=0](IteratorGetNext, sequential_14/lambda_14/strided_slice/stack, sequential_14/lambda_14/strided_slice/stack_1, sequential_14/lambda_14/strided_slice/stack_2)' with input shapes: [?,?], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.

Call arguments received:
  • inputs=tf.Tensor(shape=(None, None), dtype=float32)
  • mask=None
  • training=False

In [71]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
def return_axes(df):
    x = np.arange(df.shape[0])[:, None, None]
    y = np.arange(df.shape[1])[None, :, None]
    z = np.arange(df.shape[2])[None, None, :]
    x, y, z = np.broadcast_arrays(x, y, z)
    return x,y,z

In [ ]:
# x, y, z = return_axes(predicts)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())

In [ ]:
# x, y, z = return_axes(test)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())